In [1]:
import pandas as pd
import sqlite3
import pandas_profiling as pp

harmonized_table_file = "../../scoped-mapping/target/harmonized_table.db"

In [2]:
connection = sqlite3.connect(harmonized_table_file)

In [3]:
# # 1 minute
# bshtq = """
# select env_package
# from biosample b
# """

# bshtres = pd.read_sql_query(bshtq, connection)

# # epcounts = bshtres["env_package"].value_counts()

In [4]:
# epcounts = (
#     bshtres["env_package"]
#     .value_counts()
#     .rename_axis("env_package")
#     .reset_index(name="count")
# )
# epcounts

In [5]:
# epcounts.to_clipboard()

In [6]:
# bshtcols = list(bshtres.columns)
# bshtcols

In [7]:
# hg_candidates = pd.read_clipboard()

In [8]:
# hg_candidates

In [100]:
package_targeting = pd.read_csv("package_targeting.tsv", sep="\t")
package_targeting.fillna("", inplace=True)
package_targeting

,env_package,count,flag,repaired
0,host-associated,69981,,host-associated
1,human-gut,54078,True,human-gut
2,water,17924,,
3,built environment,14510,,
4,misc environment,14502,,
...,...,...,...,...
90,MIMARKS_16S,1,,
91,Deep-sea water,1,,
92,ocean[ENVO:00000015],1,,
93,marine sediment (ENVO:00002113),1,,


In [101]:
# target = "human-skin"

In [102]:
# target_subset = package_targeting.loc[
#     package_targeting["flag"].eq(True) & package_targeting["repaired"].eq(target)
# ]

# target_subset

,env_package,count,flag,repaired
5,human-skin,14327,True,human-skin
37,MIGS/MIMS/MIMARKS.human-skin,135,True,human-skin
58,human_skin,28,True,human-skin


In [42]:
ws_start = "('"
ws_end = "')"
ws_sep = "', '"

In [43]:
# where_string = ws_sep.join(list(target_subset["env_package"]))
# where_string = ws_start + where_string + ws_end
# where_string

"('human-skin', 'MIGS/MIMS/MIMARKS.human-skin', 'human_skin')"

omitting "gut" at 172 and "gut microbiome" at 23 (compared to "human-gut" at 54078

In [44]:
# bshtq = (
#     """select
# 	env_package,
# 	taxonomy_id,
# 	taxonomy_name,
# 	host_taxid,
# 	env_broad_scale,
# 	env_local_scale,
# 	env_medium,
# 	body_habitat,
# 	body_product,
# 	host,
# 	host_body_habitat,
# 	host_body_product,
# 	host_common_name,
# 	host_disease,
# 	host_sex,
# 	host_tissue_sampled,
# 	tissue
# from
# 	biosample b
# where
# 	--- both of the following will need to be tuned for each package
# 	env_package in """
#     + where_string
#     + """
# 	--- and taxonomy_name in ('human gut metagenome', 'Homo sapiens', 'human metagenome', 'gut metagenome')
# 	and (host_taxid = 9606 or host_taxid is null)
# 	--- or host_taxid is null or blank?"""
# )

# # bshtq

# bshtres = pd.read_sql_query(bshtq, connection)

In [45]:
# bshtres

In [46]:
# print(bshtq)

In [47]:
# bshtres = bshtres.astype(str)

In [48]:
# bshtres.describe()

In [49]:
# bshtres.to_clipboard(index=False)

In [50]:
# mine_these = pd.read_clipboard()

In [51]:
# mine_these

In [52]:
# col_list = ', '.join(list(mine_these["mine_these"]))
# # where_string = ws_start + where_string + ws_end
# print(col_list)

In [53]:
# report_title = target + " Profiling Report"
# report_title

'human-skin Profiling Report'

In [54]:
# eda = pp.ProfileReport(bshtres, title=report_title)

In [55]:
# profile_file = target + "_profile.html"
# profile_file

'human-skin_profile.html'

In [56]:
# eda.to_file(profile_file)

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# display(eda)

In [81]:
raw_repaireds = list(set(package_targeting["repaired"]))
raw_repaireds

['',
 'human-oral',
 'human-gut',
 'host-associated',
 'human-associated',
 'human-vaginal',
 'human-skin',
 'gut']

In [82]:
valid_repaireds = [x for x in raw_repaireds if pd.isnull(x) == False]
valid_repaireds.sort()
valid_repaireds

['',
 'gut',
 'host-associated',
 'human-associated',
 'human-gut',
 'human-oral',
 'human-skin',
 'human-vaginal']

In [108]:
# package_targeting["flag"] == True &

for i in valid_repaireds:
    print(i)
    #     target_subset = package_targeting.loc[package_targeting["repaired"].eq(i)]

    target_subset = package_targeting.loc[
        package_targeting["flag"].eq(True) & package_targeting["repaired"].eq(i)
    ]
    if len(target_subset.index) > 0:
        #         print(target_subset)
        where_string = ws_sep.join(list(target_subset["env_package"]))
        where_string = ws_start + where_string + ws_end
        where_string

        bshtq = (
            """select
            env_package,
            taxonomy_id,
            taxonomy_name,
            host_taxid,
            env_broad_scale,
            env_local_scale,
            env_medium,
            body_habitat,
            body_product,
            host,
            host_body_habitat,
            host_body_product,
            host_common_name,
            host_disease,
            host_sex,
            host_tissue_sampled,
            tissue
        from
            biosample b
        where
            --- both of the following will need to be tuned for each package
            env_package in """
            + where_string
            + """
            --- and taxonomy_name in ('human gut metagenome', 'Homo sapiens', 'human metagenome', 'gut metagenome')
            and (host_taxid = 9606 or host_taxid is null)
            --- or host_taxid is null or blank?"""
        )

        bshtres = pd.read_sql_query(bshtq, connection)

        print(bshtres)

        bshtres = bshtres.astype(str)

        report_title = i + " Profiling Report"
        print(report_title)

        eda = pp.ProfileReport(bshtres, title=report_title)

        profile_file = i + "_profile.html"
        print(profile_file)

        eda.to_file(profile_file)


gut
        env_package taxonomy_id      taxonomy_name host_taxid env_broad_scale  \
0               gut       10116  Rattus norvegicus       None            None   
1               gut       10116  Rattus norvegicus       None            None   
2               gut       10116  Rattus norvegicus       None            None   
3               gut       10116  Rattus norvegicus       None            None   
4               gut       10116  Rattus norvegicus       None            None   
..              ...         ...                ...        ...             ...   
190  gut microbiome       10090       Mus musculus       None            None   
191  gut microbiome       10090       Mus musculus       None            None   
192  gut microbiome       10090       Mus musculus       None            None   
193  gut microbiome       10090       Mus musculus       None            None   
194  gut microbiome       10090       Mus musculus       None            None   

    env_local_scale en

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

human-vaginal
        env_package taxonomy_id             taxonomy_name host_taxid  \
0     Human-vaginal     1632839  human vaginal metagenome       None   
1     Human-vaginal     1632839  human vaginal metagenome       None   
2     Human-vaginal     1632839  human vaginal metagenome       None   
3     Human-vaginal     1632839  human vaginal metagenome       None   
4     Human-vaginal     1632839  human vaginal metagenome       None   
...             ...         ...                       ...        ...   
1114  human-vaginal     1632839  human vaginal metagenome       None   
1115  human-vaginal     1632839  human vaginal metagenome       None   
1116  human-vaginal     1632839  human vaginal metagenome       None   
1117  human-vaginal     1632839  human vaginal metagenome       9606   
1118  human-vaginal     1632839  human vaginal metagenome       9606   

             env_broad_scale           env_local_scale  \
0          terrestrial biome  human-associated habitat   
1    

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]